In [1]:
# from coco_dataloader import COCO_loader

# cl = COCO_loader()


In [2]:
from models import HW4Net1, HW4Net2

import numpy as np
import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = (8.0, 10.0)
import os
import cv2

from tqdm import tqdm

In [3]:
# # get all images containing given categories, select one at random
# categories = ['airplane', 'bus', 'cat', 'dog', 'pizza']

# # def get_plots(image_list):
# data_path = '/mnt/cloudNAS4/akshita/Documents/datasets/coco_custom'
# num_images = 3
# all_images = []
# for category in categories:
#     cat_images = []
#     path = os.path.join(data_path, category, 'train')
#     image_list = os.listdir(path)[:num_images]
#     # print(image_list)
#     for img in image_list:
#         img = cv2.imread(os.path.join(path, img))
#         cat_images.append(img)
#     tot_images = np.concatenate(cat_images, axis=1)

#     all_images.append(tot_images)

# all_images = np.concatenate(all_images, axis=0)

# cv2.imwrite("results/sampled_images.png", all_images)

In [4]:
from dataset import MyDataset
from torch.utils.data import DataLoader
import torch
import torch.nn as nn

torch.manual_seed(60146)

categories = ['airplane', 'bus', 'cat', 'dog', 'pizza']
data_path = '/mnt/cloudNAS4/akshita/Documents/datasets/coco_custom'
train_dataset = MyDataset(root=data_path, categories=categories, split='train')

batch_size = 16
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


def train(net, train_dataloader, epochs=10):
    device = 'cuda' if torch.cuda.is_available() == True else 'cpu'
    net.train()
    net = net.to(device=device)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=1e-3, betas=(0.9, 0.99))

    tot_loss = []
    for epoch in tqdm(range(epochs)):
        running_loss = 0.0
        for i, data in enumerate(train_dataloader):
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            if (i+1)% 100 == 0:
                print("[epoch]: %d, batch: %5d] loss: %.3f" %(epoch+1, i+1, running_loss / 100))
                tot_loss.append(running_loss)
                running_loss = 0.0
    return tot_loss

net = HW4Net1()
loss_1 = train(net=net, train_dataloader=train_dataloader, epochs=10)
torch.save(net.state_dict(), 'HW4/results/net1.pt')
# np.save('first_net.npy', np.array(loss_1), allow_pickle=True)


  0%|          | 0/10 [00:00<?, ?it/s]

[epoch]: 1, batch:   100] loss: 1.369
[epoch]: 1, batch:   200] loss: 1.154
[epoch]: 1, batch:   300] loss: 1.090
[epoch]: 1, batch:   400] loss: 1.063


 10%|█         | 1/10 [00:14<02:07, 14.22s/it]

[epoch]: 2, batch:   100] loss: 0.938
[epoch]: 2, batch:   200] loss: 0.966
[epoch]: 2, batch:   300] loss: 0.918
[epoch]: 2, batch:   400] loss: 0.962


 20%|██        | 2/10 [00:28<01:53, 14.14s/it]

...

 80%|████████  | 8/10 [02:01<00:31, 15.62s/it]

[epoch]: 9, batch:   100] loss: 0.183
[epoch]: 9, batch:   200] loss: 0.249
[epoch]: 9, batch:   300] loss: 0.254
[epoch]: 9, batch:   400] loss: 0.225


 90%|█████████ | 9/10 [02:17<00:15, 15.53s/it]

[epoch]: 10, batch:   100] loss: 0.136
[epoch]: 10, batch:   200] loss: 0.208
[epoch]: 10, batch:   300] loss: 0.194
[epoch]: 10, batch:   400] loss: 0.194


100%|██████████| 10/10 [02:32<00:00, 15.21s/it]


In [5]:
net = HW4Net2()
loss_2 = train(net=net, train_dataloader=train_dataloader, epochs=10)
torch.save(net.state_dict(), 'HW4/results/net2.pt')


  0%|          | 0/10 [00:00<?, ?it/s]

[epoch]: 1, batch:   100] loss: 1.378
[epoch]: 1, batch:   200] loss: 1.130
[epoch]: 1, batch:   300] loss: 1.090
[epoch]: 1, batch:   400] loss: 1.029


 10%|█         | 1/10 [00:16<02:29, 16.56s/it]

[epoch]: 2, batch:   100] loss: 0.994
[epoch]: 2, batch:   200] loss: 0.954
[epoch]: 2, batch:   300] loss: 0.958
[epoch]: 2, batch:   400] loss: 0.982


 20%|██        | 2/10 [00:36<02:29, 18.68s/it]

...

[epoch]: 8, batch:   100] loss: 0.271
[epoch]: 8, batch:   200] loss: 0.315
[epoch]: 8, batch:   300] loss: 0.335
[epoch]: 8, batch:   400] loss: 0.346


 80%|████████  | 8/10 [02:55<00:46, 23.01s/it]

[epoch]: 9, batch:   100] loss: 0.190
[epoch]: 9, batch:   200] loss: 0.232
[epoch]: 9, batch:   300] loss: 0.254
[epoch]: 9, batch:   400] loss: 0.264


 90%|█████████ | 9/10 [03:19<00:23, 23.44s/it]

[epoch]: 10, batch:   100] loss: 0.127
[epoch]: 10, batch:   200] loss: 0.176
[epoch]: 10, batch:   300] loss: 0.232


In [ ]:
np.save('first_net.npy', np.array(loss_1), allow_pickle=True)
np.save('second_net.npy', np.array(loss_2), allow_pickle=True)

# b = np.load('a.npy', allow_pickle=True)

In [ ]:
plt.plot(loss_1, label='Net1')
plt.plot(loss_2, label='Net2')
plt.legend()
plt.show()

$Input: (B, C_{in}, H_{in}, W_{in})$ 

$Output: (B, C_{out}, H_{out}, W_{out})$

$$H_{out} = floor\left( \frac{H_{in} + 2\times pad[0] - dilation[0] \times (kernel - 1) -1 }{stride[0]} +1 \right)$$
$$W_{out} = floor\left( \frac{W_{in} + 2\times pad[1] - dilation[1] \times (kernel - 1) -1 }{stride[1]} +1 \right)$$

In *Net1*, ```dilation``` and ```stride``` are set to default values, i.e., both 1 and ```pad = 0``` . Above equations are transformed to:

$$H_{out} = floor\left( H_{in} - kernel + 1 \right)$$
$$W_{out} = floor\left( W_{in} - kernel + 1 \right)$$

In *Net2*, ```dilation``` and ```stride``` are set to default values, i.e., both 1 and ```pad = 1``` . Above equations are transformed to:

$$H_{out} = floor\left( H_{in} + 2 - kernel + 1 \right) = floor\left( H_{in}  - kernel + 3 \right)$$
$$W_{out} = floor\left( W_{in} + 2 - kernel + 1 \right) = floor\left( W_{in}  - kernel + 3 \right)$$

